In [3]:
import cv2
import os
from math import ceil
import time
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import random
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation
import matplotlib as mpl

# mpl.rcParams['animation.ffmpeg_path_vid_vid_vid_vid'] = r'C:\\ffmpeg\\bin\\ffmpeg.exe'
import matplotlib.gridspec as gridspec

from koniq_api.lib import MyModel

import os
import sys
import pandas

2022-04-15 00:19:25.788290: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/alexkkir/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2022-04-15 00:19:25.788325: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Loaded Kuti


In [4]:
MODEL = MyModel()

Loading model InceptionResNetV2


2022-04-15 00:19:30.156062: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-15 00:19:30.156109: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alexkkir-Extensa-215-51G): /proc/driver/nvidia/version does not exist
2022-04-15 00:19:30.157058: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
class VideoReader:
    def __init__(self, path):
        self.path = path
        self.cap = cv2.VideoCapture(path)
        self.height = int(self.cap.get(4))
        self.width = int(self.cap.get(3))
        self.it = iter(self)
        self.length = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)

    def __iter__(self):
        while self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret == False:
                break
            # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            yield frame

    def __call__(self):
        return next(self.it)

    def __del__(self):
        self.cap.release()

    def refresh(self):
        self.cap = cv2.VideoCapture(self.path)

In [15]:
MAX = 2

class MetricArray:
    def __init__(self, vid_path, mask_path, n_height, n_width):
        self.vid = None if vid_path is None else VideoReader(vid_path)
        self.n_height = n_height
        self.n_width = n_width
        self.block_height = self.vid.height // n_height
        self.block_width = self.vid.width // n_width
        self.scores = np.zeros((self.vid.length, self.n_height, self.n_width))
        self.mask_raw = None if mask_path is None else VideoReader(mask_path)
        self.mask = np.zeros_like(self.scores)
        self.length = self.vid.length
        self.metric = MODEL

    def get_subframe(self, frame, i, j) -> np.array:
        return frame[
               i * self.block_height: min(self.vid.height, (i + 1) * self.block_height),
               j * self.block_width: min(self.vid.width, (j + 1) * self.block_width)
               ]

    def split_video(self):
        """
        devide video on n * m subvideos and save each of them in ./tmp/ folder
        :return:
        """

        TMP_DIR = './tmp/'
        if not os.path.exists(TMP_DIR): 
            os.mkdir(TMP_DIR)
        for f in os.listdir(TMP_DIR):
            os.remove(TMP_DIR + f)

        fourcc = cv2.VideoWriter_fourcc(*'XVID')

        for i in range(self.n_height):
            for j in range(self.n_width):
                out = cv2.VideoWriter(f'{TMP_DIR}{i}_{j}.mp4', fourcc, vid.fps, (self.block_width, self.block_height))
                for frame in vid:
                    subframe = self.get_subframe(frame, i, j)
                    out.write(subframe)
                out.release()
                vid.refresh()
                
    
    def calc_raw_metric(self):
        for frame_idx, frame in tqdm(enumerate(self.vid), total=self.length):
            # if frame_idx > MAX:
            #     return
            for i in range(self.n_height):
                for j in range(self.n_width):
                    subframe = self.get_subframe(frame, i, j)
                    score = MODEL(subframe)
                    self.scores[frame_idx][i][j] = score
    
    def resize_mask(self):
        self.mask_raw.refresh()
        for fi, frame in enumerate(self.mask_raw):
            for i in range(self.mask.shape[1]):
                for j in range(self.mask.shape[2]):
                    self.mask[fi][i][j] = frame[ 
                        i * self.block_height : (i + 1) * (self.block_height ), 
                        j * self.block_width: (j + 1) * (self.block_width)].mean() / 255     
        self.mask_raw.refresh()


    def apply_mask(self):
        return self.scores * self.mask
    
    def calc_metric(self):
        return self.apply_mask().mean()

In [17]:
m = MetricArray(folder + videos[0], None, 1, 1)
sys.getsizeof(m)

48

In [18]:
file = 'arrays_hair_full.npy'
folder = 'hair_x265/'
videos = sorted(os.listdir(folder)) 

m = MetricArray(folder + videos[0], None, 1, 1)

for video in videos:
    m.vid = VideoReader(folder + video)
    print(video)
    m.calc_raw_metric()
    with open(file, 'ab') as f:
        np.save(f, m.scores)

enc_res_kingsoft_v1_mv_offline_2k_v1_blue_hair_33.mp4


  0%|          | 0/300 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
file = 'arrays_boys_full.npy'
folder = 'boys_x265/'
videos = sorted(os.listdir(folder)) 

for video in videos:
    print(video)
    m = MetricArray(folder + video, None, 1, 1)
    m.calc_raw_metric()
    with open(file, 'ab') as f:
        np.save(f, m.scores)

In [11]:
with open(file, 'rb') as f:
    x1 = np.load(f)

print(x1)

[[[12.62037468]]

 [[10.01094913]]

 [[11.35167313]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.        ]]

 [[ 0.    

In [ ]:
folder = 'blue_hair_x265'

videos = sorted(os.listdir(folder))
bscores = []
subjs = get_subj_scores(videos)

with open(file, 'rb') as f:
    try:
        while true:
            bscores.append(np.load(f))
            
assert len(videos) == len(bscores) == len(subjs)